In [50]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv
import time

In [5]:
def get_links(url):
    """
    Returns a list of internal links on a Wikipedia page given its URL
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    list_url = []
    for link in links:
        href = link.get('href')
        if href is not None and (href.startswith("/wiki/") or "id.wikipedia.org/wiki/" in href):
            if not(href in url_list):
                list_url.append(href)
                url_list.add(href)
    return list_url

In [41]:
def scrape_table(url):
    """
    Scrapes the first table found on a Wikipedia page given its URL
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', class_='wikitable')
    if table is None:
        return None
    else:
        df = pd.read_html(str(table))[0]
        df.columns = df.columns.map(lambda x: str(x[0]) if isinstance(x, tuple) else str(x))
        # remove any columns that have no header
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        # merge multi-level column headers into a single header
        df.columns = [' '.join(col).strip() for col in df.columns.values]
        return df

In [58]:
def save_table(df, filename):
    """
    Saves a Pandas DataFrame as CSV, TSV, and HTML files
    """
    # save the table in CSV format
    csv_filename = f"csv_data/{filename}.csv"
    df.to_csv(csv_filename)

In [14]:
import re

def has_extension_or_pattern(string):
    if re.search(r'\.[a-zA-Z0-9]+$', string):
        return True
    elif re.search(r'/wiki/(\w+):', string):
        return True
    else:
        return False

In [63]:
# set up the initial URL to crawl
start_url = "https://id.wikipedia.org/wiki/Kucing"

# set up a list to keep track of all visited URLs
url_list = set()
url_list.add(start_url)

# set up a queue of URLs to visit
url_queue = []
url_queue.append(start_url)
table_counter = 0

In [64]:
# loop through all URLs in the queue
while len(url_queue) > 0:
    # get the next URL to visit
    url = url_queue.pop(0)
    print(url)

    
    max_retries = 5

    for i in range(max_retries):
        try:
            df = scrape_table(url)

            # if a table was found, save it to disk
            if df is not None:
                # extract the title of the Wikipedia page to use as the filename
                title = url.split('/')[-1].replace('_', ' ')
                save_table(df, title)
                table_counter = table_counter + 1
                print(table_counter)


            # get all internal links on the page and add them to the queue
            links = get_links(url)
            for link in links:
                full_link = f"https://id.wikipedia.org{link}"
                if (full_link not in url_list) and not(has_extension_or_pattern(full_link)):
                    url_list.add(full_link)
                    url_queue.append(full_link)
            break
        except requests.exceptions.ConnectionError:
            print(f"Connection error occurred. Retrying in 5 seconds ({i+1}/{max_retries})...")
            time.sleep(5)

https://id.wikipedia.org/wiki/Kucing
https://id.wikipedia.org/wiki/Halaman_Utama
https://id.wikipedia.org/wiki/Kuching
1
https://id.wikipedia.org/wiki/Hewan
2
https://id.wikipedia.org/wiki/Chordata
https://id.wikipedia.org/wiki/Binatang_menyusui
https://id.wikipedia.org/wiki/Carnivora
https://id.wikipedia.org/wiki/Felidae
https://id.wikipedia.org/wiki/Felis
https://id.wikipedia.org/wiki/Carolus_Linnaeus
https://id.wikipedia.org/wiki/Felis_silvestris
https://id.wikipedia.org/wiki/Felis_catus
https://id.wikipedia.org/wiki/Jantan
https://id.wikipedia.org/wiki/Betina
https://id.wikipedia.org/wiki/Nama_ilmiah
https://id.wikipedia.org/wiki/Mamalia
https://id.wikipedia.org/wiki/Karnivora
https://id.wikipedia.org/wiki/Familia
https://id.wikipedia.org/wiki/Kucing_besar
https://id.wikipedia.org/wiki/Singa
https://id.wikipedia.org/wiki/Harimau
https://id.wikipedia.org/wiki/Manusia
3
https://id.wikipedia.org/wiki/Kerangka
https://id.wikipedia.org/wiki/Siprus
4
https://id.wikipedia.org/wiki/Mesir_K

https://id.wikipedia.org/wiki/Wikispecies
https://id.wikipedia.org/wiki/Wayback_Machine
https://id.wikipedia.org/wiki/Felinologi
https://id.wikipedia.org/wiki/Anatomi_kucing
https://id.wikipedia.org/wiki/Genetika_kucing
https://id.wikipedia.org/wiki/Indera_kucing
https://id.wikipedia.org/wiki/Kucing_bermata_ganjil
https://id.wikipedia.org/wiki/Kucing_kerdil
https://id.wikipedia.org/wiki/Kucing_polidaktil
https://id.wikipedia.org/wiki/Kucing_tupai
https://id.wikipedia.org/wiki/Kucing_hitam
https://id.wikipedia.org/wiki/Kucing_putih
https://id.wikipedia.org/wiki/Kucing_solid
https://id.wikipedia.org/wiki/Kucing_tabi
https://id.wikipedia.org/wiki/Kucing_warna_poin
https://id.wikipedia.org/wiki/Makanan_kucing
31
https://id.wikipedia.org/wiki/Pencabutan_cakar
https://id.wikipedia.org/wiki/Peneuteran
https://id.wikipedia.org/wiki/Vaksinasi_kucing
https://id.wikipedia.org/wiki/Feline_immunodeficiency_virus
https://id.wikipedia.org/wiki/Asma_kucing
https://id.wikipedia.org/wiki/Cacing_jantung


https://id.wikipedia.org/wiki/Kucing_Geoffroy
https://id.wikipedia.org/wiki/Kodkod
https://id.wikipedia.org/wiki/Kucing_andes
https://id.wikipedia.org/wiki/Kucing_pampa
https://id.wikipedia.org/wiki/Ocelot
https://id.wikipedia.org/wiki/Oncilla
https://id.wikipedia.org/wiki/Margay
https://id.wikipedia.org/wiki/Serval
https://id.wikipedia.org/wiki/Karakal
https://id.wikipedia.org/wiki/Kucing_emas
https://id.wikipedia.org/wiki/Catopuma
https://id.wikipedia.org/wiki/Kucing_merah
https://id.wikipedia.org/wiki/Kucing_emas_Asia
https://id.wikipedia.org/wiki/Kucing_batu
https://id.wikipedia.org/wiki/Pantherinae
https://id.wikipedia.org/wiki/Neofelis
https://id.wikipedia.org/wiki/Macan_dahan
https://id.wikipedia.org/wiki/Macan_dahan_kalimantan
https://id.wikipedia.org/wiki/Panthera
https://id.wikipedia.org/wiki/Jaguar
https://id.wikipedia.org/wiki/Macan_tutul
https://id.wikipedia.org/wiki/Macan_Tutul_Salju
https://id.wikipedia.org/wiki/Wikidata
https://id.wikipedia.org/wiki/Fossilworks
https://

https://id.wikipedia.org/wiki/Korea_Selatan
https://id.wikipedia.org/wiki/Guro-gu
https://id.wikipedia.org/wiki/Batas_kota
84
https://id.wikipedia.org/wiki/Bandara_Internasional_Kuching
85
https://id.wikipedia.org/wiki/Bau,_Malaysia
https://id.wikipedia.org/wiki/Hutan_hujan_tropis
https://id.wikipedia.org/wiki/Klasifikasi_iklim_K%C3%B6ppen
86
https://id.wikipedia.org/wiki/Organisasi_Meteorologi_Dunia
https://id.wikipedia.org/wiki/Senja
https://id.wikipedia.org/wiki/Orang_Melayu
87
https://id.wikipedia.org/wiki/Etnis_Tionghoa
https://id.wikipedia.org/wiki/Iban
https://id.wikipedia.org/wiki/Bidayuh
https://id.wikipedia.org/wiki/Melanau
https://id.wikipedia.org/wiki/Buddhisme
88
https://id.wikipedia.org/wiki/Taoisme
https://id.wikipedia.org/wiki/Kristen
89
https://id.wikipedia.org/wiki/Animisme
https://id.wikipedia.org/wiki/Agama_Hindu
90
https://id.wikipedia.org/wiki/Sikhisme
https://id.wikipedia.org/wiki/Sekularisme
https://id.wikipedia.org/wiki/Klenteng
https://id.wikipedia.org/wiki/As

https://id.wikipedia.org/wiki/Taksonomi_(biologi)
https://id.wikipedia.org/wiki/Systema_Naturae
https://id.wikipedia.org/wiki/Jean-Baptiste_Lamarck
https://id.wikipedia.org/wiki/Protozoa
111
https://id.wikipedia.org/wiki/Filogenetik_molekuler
https://id.wikipedia.org/wiki/Takson
https://id.wikipedia.org/wiki/Daging
112
https://id.wikipedia.org/wiki/Susu
https://id.wikipedia.org/wiki/Telur
https://id.wikipedia.org/wiki/Kulit_(produk_hewan)
https://id.wikipedia.org/wiki/Wol
https://id.wikipedia.org/wiki/Hewan_timangan
https://id.wikipedia.org/wiki/Hewan_pekerja
https://id.wikipedia.org/wiki/Bahasa_Arab
113
https://id.wikipedia.org/wiki/Bahasa_Sanskerta
114
https://id.wikipedia.org/wiki/Bahasa_Latin
https://id.wikipedia.org/wiki/Insekta
https://id.wikipedia.org/wiki/Embrio
115
https://id.wikipedia.org/wiki/Prokariotik
https://id.wikipedia.org/wiki/Protista
https://id.wikipedia.org/wiki/Alga
https://id.wikipedia.org/wiki/Autotrof
https://id.wikipedia.org/wiki/Respirasi_aerob
https://id.wik

...

https://id.wikipedia.org/wiki/Leukoplas
https://id.wikipedia.org/wiki/Amiloplas
https://id.wikipedia.org/wiki/Nukleolus
https://id.wikipedia.org/wiki/Sitosol
https://id.wikipedia.org/wiki/Proteasom
https://id.wikipedia.org/wiki/Pautan_genetik
https://id.wikipedia.org/wiki/Mendel
https://id.wikipedia.org/wiki/Korelasi
1489
https://id.wikipedia.org/wiki/Profase
https://id.wikipedia.org/wiki/Makerel
https://id.wikipedia.org/wiki/Ikan_laut
https://id.wikipedia.org/wiki/Scombridae
https://id.wikipedia.org/wiki/Tenggiri
https://id.wikipedia.org/wiki/Ikan_kembung
https://id.wikipedia.org/wiki/Pelagis
https://id.wikipedia.org/wiki/Teluk
https://id.wikipedia.org/wiki/Ugahari
https://id.wikipedia.org/wiki/Saus_cabe
https://id.wikipedia.org/wiki/Saus_tomat
https://id.wikipedia.org/wiki/Sarden
https://id.wikipedia.org/wiki/Rastrelliger_brachysoma
https://id.wikipedia.org/wiki/Kembung_perempuan
https://id.wikipedia.org/wiki/Rastrelliger_kanagurta
https://id.wikipedia.org/wiki/Kembung_lelaki
https:/

https://id.wikipedia.org/wiki/Laut_Liguria
https://id.wikipedia.org/wiki/Laut_Norwegia
https://id.wikipedia.org/wiki/Selat_Gibraltar
https://id.wikipedia.org/wiki/Lithuania
https://id.wikipedia.org/wiki/Eropa_Tenggara
https://id.wikipedia.org/wiki/Samudra_Arctic
https://id.wikipedia.org/wiki/Kolonisasi
https://id.wikipedia.org/wiki/Spruce
https://id.wikipedia.org/wiki/Chestnut
https://id.wikipedia.org/wiki/Encyclopaedia_Britannica
https://id.wikipedia.org/wiki/Afro-Eurasia
https://id.wikipedia.org/wiki/Superbenua
1498
https://id.wikipedia.org/wiki/Gondwana
https://id.wikipedia.org/wiki/Laurasia
https://id.wikipedia.org/wiki/Pangaea
https://id.wikipedia.org/wiki/Pannotia
https://id.wikipedia.org/wiki/Rodinia
https://id.wikipedia.org/wiki/Columbia_(superbenua)
https://id.wikipedia.org/wiki/Kenorland
https://id.wikipedia.org/wiki/Nena_(superbenua)
https://id.wikipedia.org/wiki/Ur_(benua)
https://id.wikipedia.org/wiki/Vaalbara
1499
https://id.wikipedia.org/wiki/Kraton_Amazonia
https://id.w

1523
https://id.wikipedia.org/wiki/Canberra
1524
https://id.wikipedia.org/wiki/Agama_di_Australia
1525
https://id.wikipedia.org/wiki/Kekristenan_di_Australia
1526
https://id.wikipedia.org/wiki/Islam_di_Australia
1527
https://id.wikipedia.org/wiki/Agama_di_Australia#Agama_lainnya
1528
https://id.wikipedia.org/wiki/Orang_Australia
1529
https://id.wikipedia.org/wiki/Aussie
https://id.wikipedia.org/wiki/Federalisme
https://id.wikipedia.org/wiki/Monarki_Australia
1530
https://id.wikipedia.org/wiki/Charles_III_dari_Britania_Raya
1531
https://id.wikipedia.org/wiki/Gubernur_Jenderal_Australia
https://id.wikipedia.org/wiki/David_Hurley
https://id.wikipedia.org/wiki/Perdana_Menteri_Australia
https://id.wikipedia.org/wiki/Anthony_Albanese
https://id.wikipedia.org/wiki/Parlemen_Australia
https://id.wikipedia.org/wiki/Senat_Australia
https://id.wikipedia.org/wiki/Dewan_Perwakilan_Australia
https://id.wikipedia.org/wiki/Federasi_Australia
1532
https://id.wikipedia.org/wiki/Undang-Undang_Australia_19

https://id.wikipedia.org/wiki/1988
1558
https://id.wikipedia.org/wiki/Dunia_Persemakmuran
1559
https://id.wikipedia.org/wiki/Daftar_Gubernur_Jenderal_Australia
https://id.wikipedia.org/wiki/Partai_Buruh_Australia
1560
https://id.wikipedia.org/wiki/Koalisi_(Australia)
1561
https://id.wikipedia.org/wiki/Partai_Liberal_Australia
1562
https://id.wikipedia.org/wiki/Partai_Nasional_Australia
https://id.wikipedia.org/wiki/Partai_Hijau_Australia
https://id.wikipedia.org/wiki/Julia_Gillard
1563
https://id.wikipedia.org/wiki/Pembagian_administratif_Australia
https://id.wikipedia.org/wiki/Daftar_negara_bagian_dan_teritorial_di_Australia
https://id.wikipedia.org/wiki/Gubernur-Jenderal_Australia
https://id.wikipedia.org/wiki/Daftar_ibu_kota_Australia
https://id.wikipedia.org/wiki/Pulau_Norfolk
https://id.wikipedia.org/wiki/Kingston,_Pulau_Norfolk
https://id.wikipedia.org/wiki/Darwin,_Wilayah_Utara
1564
https://id.wikipedia.org/wiki/Hobart
https://id.wikipedia.org/wiki/Victoria_(Australia)
https://i

ValueError: invalid literal for int() with base 10: '2;'

In [56]:
df = scrape_table("https://id.wikipedia.org/wiki/Daftar_negara_berdaulat")

In [60]:
df

,N a m a d a l a m b a h a s a I n d o n e s i a,N a m a d a l a m b a h a s a r e s m i,K e t e r a n g a n
0,Afganistan – Republik Islam Afganistan,Pashto: د افغانستان اسلامي جمهوریت Dari: جمهور...,Secara de facto dikontrol oleh Keamiran Islam ...
1,Afrika Selatan – Republik Afrika Selatan[3],Inggris: South Africa – Republic of South Afri...,NaN
2,Afrika Tengah – Republik Afrika Tengah[4],Prancis: Centrafrique - République Centrafrica...,NaN
3,Albania – Republik Albania,Albania: Shqipëria – Republika e Shqipërise In...,NaN
4,Aljazair – Republik Demokratis Rakyat Aljazair,Arab: الجمهورية الجزائرية الديمقراطية الشعبية ...,NaN
...,...,...,...
213,Yordania – Kerajaan Hasyimiyah Yordania,Arab: المملكة الأردنّيّة الهاشميّة – الأردن → ...,NaN
214,Yunani – Republik Hellenik,Yunani: Ελλάδα – Ελληνική Δημοκρατία → Elláda ...,NaN
215,Z,Z,Z
216,Zambia – Republik Zambia,Inggris: Zambia – Republic of Zambia,NaN


In [61]:
if df is not None:
    # extract the title of the Wikipedia page to use as the filename
    title = url.split('/')[-1].replace('_', ' ')
    save_table(df, title)
    table_counter = table_counter + 1
    print(table_counter)

460


In [62]:
x = pd.read_csv(rf"csv_data/{title}.csv")
x

,Unnamed: 0,N a m a d a l a m b a h a s a I n d o n e s i a,N a m a d a l a m b a h a s a r e s m i,K e t e r a n g a n
0,0,Afganistan – Republik Islam Afganistan,Pashto: د افغانستان اسلامي جمهوریت Dari: جمهور...,Secara de facto dikontrol oleh Keamiran Islam ...
1,1,Afrika Selatan – Republik Afrika Selatan[3],Inggris: South Africa – Republic of South Afri...,NaN
2,2,Afrika Tengah – Republik Afrika Tengah[4],Prancis: Centrafrique - République Centrafrica...,NaN
3,3,Albania – Republik Albania,Albania: Shqipëria – Republika e Shqipërise In...,NaN
4,4,Aljazair – Republik Demokratis Rakyat Aljazair,Arab: الجمهورية الجزائرية الديمقراطية الشعبية ...,NaN
...,...,...,...,...
213,213,Yordania – Kerajaan Hasyimiyah Yordania,Arab: المملكة الأردنّيّة الهاشميّة – الأردن → ...,NaN
214,214,Yunani – Republik Hellenik,Yunani: Ελλάδα – Ελληνική Δημοκρατία → Elláda ...,NaN
215,215,Z,Z,Z
216,216,Zambia – Republik Zambia,Inggris: Zambia – Republic of Zambia,NaN


In [65]:
import pickle

with open("url_list.pickle", "wb") as f:
    pickle.dump(url_list,f)

with open("url_queue.pickle", "wb") as f:
    pickle.dump(url_queue,f)

with open("table.pickle", "wb") as f:
    pickle.dump(table_counter,f)